In [ ]:
import cv2
from ultralytics import YOLO

# Load the model
yolo = YOLO('yolov8s.pt')

# Load the video capture
videoCap = cv2.VideoCapture(0)

# Function to get class colors
def getColours(cls_num):
    base_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    color_index = cls_num % len(base_colors)
    increments = [(1, -2, 1), (-2, 1, -1), (1, -1, 2)]
    color = [base_colors[color_index][i] + increments[color_index][i] * 
    (cls_num // len(base_colors)) % 256 for i in range(3)]
    return tuple(color)


while True:
    ret, frame = videoCap.read()
    if not ret:
        continue
    results = yolo.track(frame, stream=True)


    for result in results:
        # get the classes names
        classes_names = result.names

        # iterate over each box
        for box in result.boxes:
            # check if confidence is greater than 40 percent
            if box.conf[0] > 0.4:
                # get coordinates
                [x1, y1, x2, y2] = box.xyxy[0]
                # convert to int
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # get the class
                cls = int(box.cls[0])

                # get the class name
                class_name = classes_names[cls]

                # get the respective colour
                colour = getColours(cls)

                # draw the rectangle
                cv2.rectangle(frame, (x1, y1), (x2, y2), colour, 2)

                # put the class name and confidence on the image
                cv2.putText(frame, f'{classes_names[int(box.cls[0])]} {box.conf[0]:.2f}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, colour, 2)
                
    # show the image
    cv2.imshow('frame', frame)

    # break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the video capture and destroy all windows
videoCap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'yolov5.pt'

In [7]:
# Ruta específica a modificar
ruta = r"C:\Users\chj06\OneDrive\Documents\Escuela\7_semestre\Sistemas_Embebidos\yolov5\trafic_signals-1\data.yaml"

# Convertir las barras invertidas (\) a barras normales (/)
ruta_actualizada = ruta.replace("\\", "/")

print("Ruta original:")
print(ruta)
print("\nRuta actualizada:")
print(ruta_actualizada)


Ruta original:
C:\Users\chj06\OneDrive\Documents\Escuela\7_semestre\Sistemas_Embebidos\yolov5\trafic_signals-1\data.yaml

Ruta actualizada:
C:/Users/chj06/OneDrive/Documents/Escuela/7_semestre/Sistemas_Embebidos/yolov5/trafic_signals-1/data.yaml


In [ ]:
# Entrenar el modelo
!python train.py --img 640 --batch 16 --epochs 20 --data trafic_signals-1/data.yaml --weights yolov5s.pt


In [10]:
!python export.py --weights runs/train/exp/weights/best.pt --img 640 --batch 1


Could not find platform independent libraries <prefix>
export: data=C:\Users\chj06\OneDrive\Documents\Escuela\7_semestre\Sistemas_Embebidos\yolov5\data\coco128.yaml, weights=['runs/train/exp/weights/best.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript']
YOLOv5  v7.0-392-gf003c3df Python-3.12.3 torch-2.5.1+cpu CPU

Traceback (most recent call last):
  File "c:\Users\chj06\OneDrive\Documents\Escuela\7_semestre\Sistemas_Embebidos\yolov5\export.py", line 1546, in <module>
    main(opt)
  File "c:\Users\chj06\OneDrive\Documents\Escuela\7_semestre\Sistemas_Embebidos\yolov5\export.py", line 1541, in main
    run(**vars(opt))
  File "c:\Users\chj06\OneDrive\Documents\Escuela\7_semestre\Sistemas_Embebidos

In [17]:
import os
import shutil
import random

def split_images(dataset_dir, output_dir, split_ratio=0.7):
    """
    Divide las imágenes en conjuntos de entrenamiento y validación.

    Args:
        dataset_dir (str): Ruta al directorio que contiene todas las imágenes.
        output_dir (str): Directorio donde se crearán las carpetas train/val.
        split_ratio (float): Proporción de datos para entrenamiento (e.g., 0.7 para 70/30).
    """
    # Crear rutas de salida
    train_img_dir = os.path.join(output_dir,'train')
    val_img_dir = os.path.join(output_dir, 'val')

    # Crear carpetas necesarias
    for dir_path in [train_img_dir, val_img_dir]:
        os.makedirs(dir_path, exist_ok=True)

    # Listar imágenes
    images = [f for f in os.listdir(dataset_dir) if f.endswith(('.jpg', '.png'))]

    # Barajar y dividir en conjuntos
    random.shuffle(images)
    split_index = int(len(images) * split_ratio)
    train_images = images[:split_index]
    val_images = images[split_index:]

    # Mover archivos
    for img_file in train_images:
        shutil.copy(os.path.join(dataset_dir, img_file), os.path.join(train_img_dir, img_file))

    for img_file in val_images:
        shutil.copy(os.path.join(dataset_dir, img_file), os.path.join(val_img_dir, img_file))

    print(f"Conjunto dividido: {len(train_images)} entrenamiento, {len(val_images)} validación.")

# Parámetros del script
dataset_dir = r'./archive\Direction\Up'  # Cambia esto por la ruta a tu dataset
output_dir = './dataset/arriba'  # Cambia esto por la ruta donde guardarás los datos

# Ejecutar la división
split_images(dataset_dir, output_dir)


Conjunto dividido: 644 entrenamiento, 277 validación.
